In [12]:
!pip install yfinance

from typing import Dict, List, Tuple
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as pkl
import seaborn as sns
import requests
import tensorflow as tf
import time
import yfinance as yf

print(f'TensorFlow version: {tf.__version__}')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
TensorFlow version: 2.11.0


# Functions From Exploration Notebook

In [39]:
def fetch_yahoo_finance_market_chart(coin_code: str,
                                     vs_currency: str,
                                     period: str = '5y') -> pd.DataFrame:
    ticker = yf.Ticker(f'{coin_code}-{vs_currency}')
    df = ticker.history(period=period,    # Last 5 years
                        interval='1d',  # Daily data
                        prepost=False,  # Don't fetch pre-post market data
                        keepna=True,    # Whether to let yfinance drop rows with NaN
                        timeout=None,
                        raise_errors=True)

    # Returns pd.DataFrame with columns:
    #   ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
    # 'Dividends' and 'Stock Splits' aren't useful (all 0s in df.describe())
    return df.drop(columns=['Dividends', 'Stock Splits'])

In [14]:
class ZScaler:

    def __init__(self,
                 df: pd.DataFrame,
                 scale_columns: List[str] = None,
                 label_columns: List[str] = None):
        """
        :param df: pandas.DataFrame to transform/inverse-transform w.r.t.
        :param label_columns: ordered list of columns that need to be
                              de-normalised from model output tensor.
        """
        self.mean = df.mean()
        self.std = df.std()
        self.scale_columns = scale_columns
        self.scale_column_indices = {name: i
                                     for i, name in enumerate(df.columns)
                                     if name in scale_columns}
        self.label_columns = label_columns

    def transform(self,
                  df: pd.DataFrame) -> np.array:
        """
        Scales the given pandas.DataFrame by calculating z-scores on each column
        with the mean and standard deviation of the training data.

        :param df: pandas.DataFrame to normalise.
        :return: column-wise z-normalised np.array.
        """
        df = df.copy()
        df[self.scale_columns] -= self.mean[self.scale_columns]
        df[self.scale_columns] /= self.std[self.scale_columns]
        return df.values

    def inverse_transform(self,
                          tensor: tf.Tensor,
                          is_input: bool = False) -> np.array:
        """
        Inverse scales the model output w.r.t. the label columns or `columns` if
        given. This is an in-place operation.

        :param outputs: tensor of shape (_, label_steps, label_features)
                        containing model outputs.
        :param is_input: whether the tensor is of the input format so that we
                         scale w.r.t the scale columns instead of label columns.
        """
        if is_input:
            for column_name, i in self.scale_column_indices.items():
                tensor[:, i] *= self.std[column_name]
                tensor[:, i] += self.mean[column_name]
        else:
            for i, label_name in enumerate(self.label_columns):
                # For all outputs in this label column, inverse z-score transform
                # the values.
                tensor[:, i] *= self.std[label_name]
                tensor[:, i] += self.mean[label_name]

In [15]:
class CryptoDataset:

    """
    Adapted from https://www.tensorflow.org/tutorials/structured_data/time_series#data_windowing
    """

    def __init__(self,
                 training_df: pd.DataFrame,
                 validation_df: pd.DataFrame,
                 testing_df: pd.DataFrame,
                 input_steps: int = 14,
                 label_steps: int = 1,
                 offset: int = 1,
                 label_columns: List[str]=None,
                 scale_columns: List[str]=None,
                 batch_size: int = 32,
                 transform: bool = True):
        """
        Adapted from https://www.tensorflow.org/tutorials/structured_data/time_series#1_indexes_and_offsets

        :param df: pandas.DataFrame to split into windowed training, validation
                   and testing datasets.
        :param training_split: decimal percentage of `df` rows to allocate for
                               training.
        :param validation_split: decimal percentage of `df` rows after removing
                                 training rows to allocate for validation.
        :param input_steps: number of time steps for the input window.
        :param label_steps: number of time steps for the label window (to
                            generate predictions for).
        :param offset: number of time steps after the input time steps.
        :param label_columns: ordered list of column names to predict and match
                              against the columns of the model output. If None,
                              all columns are predicted.
        :param batch_size: number of (input, label) pairs in a batch.
        :param transform: whether to scale data using z-scores. Only disable
                          for illustrative purposes.
        """
        self.training_df = training_df
        self.validation_df = validation_df
        self.testing_df = testing_df

        # For financial data with columns that have continuous, large-range, or
        # unknown distributions, we should always scale down those columns to
        # reduce the gradient used when updating weights during gradient
        # descent, which ensures stable model training.
        self.zscaler = ZScaler(training_df,
                               scale_columns=scale_columns,
                               label_columns=label_columns)
        self.transform = transform

        # pandas.DataFrame gets converted into Tensors for model training so we
        # need to log column indices.
        self.column_indices = {name: i for i, name
                               in enumerate(training_df.columns)}
        self.label_columns = label_columns
        # model.predict() returns Tensor so need to record label indices.
        if label_columns is not None:
            self.label_column_indices = {name: i for i, name
                                          in enumerate(label_columns)}

        self.input_steps = input_steps
        self.label_steps = label_steps
        self.offset = offset

        self.total_window_size = input_steps + offset

        self.input_slice = slice(0, input_steps)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start_idx = self.total_window_size - self.label_steps
        self.label_slice = slice(self.label_start_idx, None)
        self.label_indices = np.arange(self.total_window_size)[self.label_slice]

        self.batch_size = batch_size

    def __repr__(self):
        return f'Total window size: {self.total_window_size}\n' \
               f'Input step indices: {self.input_indices}\n' \
               f'Label step indices: {self.label_indices}\n' \
               f'Label column names: {self.label_columns}'

    def split_window(self,
                     window: tf.Tensor):
        """
        Splits a batch of windows of time series data into inputs and labels.
        Adapted from https://www.tensorflow.org/tutorials/structured_data/time_series#2_split

        :param window: tensor of shape (batch_size, total_window_size, input_features).
        :return inputs: data with all columns from input time steps.
        :return labels: data with label columns from label time steps.
        """
        # For all batch items, `self.input_slice` time steps are the inputs.
        inputs = window[:, self.input_slice, :]

        # For all batch items, `self.label_slice` time steps are the labels.
        labels = window[:, self.label_slice, :]
        # `labels` should only consist of columns given in `self.label_columns`.
        if self.label_columns is not None:
            # Get the label columns in every window in every batch item and
            # stack them into another tensor.
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name
                               in self.label_columns],
                              axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually to makes `tf.data.Dataset`s easier to inspect.
        inputs.set_shape([None, self.input_steps, None])
        labels.set_shape([None, self.label_steps, None])

        return inputs, labels


    def _make_dataset(self,
                      df: pd.DataFrame) -> tf.data.Dataset:
        """
        Converts a time series pandas.DataFrame to a tensorflow.data.Dataset.
        Adapted from https://www.tensorflow.org/tutorials/structured_data/time_series#4_create_tfdatadatasets

        :param df: pandas.DataFrame containing time series data.
        :return: tf.data.Dataset i.e. shuffled collection of batches, each
                 containing `batch_size` (input, label) pairs.
        """
        # Generate time series data (i.e. generate windows at every time step),
        # split into batches of `batch_size` windows, and shuffle.
        data = self.zscaler.transform(df) if self.transform else df.values
        ds = tf.keras.utils.timeseries_dataset_from_array(data,
                                                          None,
                                                          self.total_window_size,
                                                          sequence_stride=1,
                                                          shuffle=False,
                                                          batch_size=self.batch_size)
        # Split each batch of windows into (inputs, labels) pairs.
        ds = ds.map(self.split_window)
        return ds

    @property
    def training_dataset(self):
        return self._make_dataset(self.training_df)

    @property
    def validation_dataset(self):
        return self._make_dataset(self.validation_df)

    @property
    def testing_dataset(self):
        return self._make_dataset(self.testing_df)

    @property
    def example(self):
        """
        Get and cache an example batch of `(inputs, labels)` for plotting.
        """
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from `self.testing_dataset`
            result = next(iter(self.testing_dataset))
            # Cache it for next time
            self._example = result
        return result

    def plot(self,
             model: tf.keras.Model = None,
             plot_col: str = 'Close',
             max_subplots: int = 3,
             dataset_type: str = 'testing',
             inverse_transform: bool = None):
        """
        Visualises the first sample of however many batches of a dataset as a
        split window.
        Adapted from https://www.tensorflow.org/tutorials/structured_data/time_series#3_plot

        :param model: Reference to the trained tensorflow.keras.Model to plot
                      predictions for comparison.
        :param plot_col: Name of column to plot.
        :param max_subplots: maximum number of subplots (examples) to plot in
                             the figure.
        :param dataset_type: prefix of dataset to plot examples from (default
                             testing)
        :param inverse_transform: whether to inverse transform the model output
                                  so that the plot is against the original
                                  un-normalised scale.
        """
        fig = plt.figure(figsize=(12, 8))
        plt.close()
        plot_col_idx = self.column_indices[plot_col]

        if 'training'.startswith(dataset_type):
            ds = self.training_dataset
        elif 'validation'.startswith(dataset_type):
            ds = self.validation_dataset
        else:
            ds = self.testing_dataset

        max_n = min(max_subplots, len(ds))
        ylabel = f'{plot_col}{"" if inverse_transform or not self.transform else " [normalised]"}'
        for i, (inputs, labels) in enumerate(ds.take(max_n)):
            ax = fig.add_subplot(max_n, 1, i + 1)
            ax.set_ylabel(ylabel)
            plot_inputs = inputs[0, :, plot_col_idx].numpy()
            if self.transform and inverse_transform:
                plot_inputs = plot_inputs[:, tf.newaxis]
                self.zscaler.inverse_transform(plot_inputs)
            ax.plot(self.input_indices,
                    plot_inputs.flatten(),
                    label='Inputs',
                    marker='.',
                    zorder=-10)
            
            if self.label_columns:
                label_col_idx = self.label_column_indices.get(plot_col, None)
            else:
                label_col_idx = plot_col_idx
            
            if label_col_idx is None:
                continue

            labels = labels[0, :, label_col_idx].numpy()
            if self.transform and inverse_transform:
                labels = labels[:, tf.newaxis]
                self.zscaler.inverse_transform(labels)
            ax.scatter(self.label_indices,
                       labels.flatten(),
                       edgecolors='k',
                       label='Labels',
                       c='#2ca02c')
                       # s=64)

            if model is not None:
                prediction = model(inputs)[0]
                if self.transform and inverse_transform:
                    self.zscaler.inverse_transform(prediction)
                ax.scatter(self.label_indices,
                           prediction[:, label_col_idx],
                           marker='X',
                           edgecolors='k',
                           label='Predictions',
                           c='#ff7f0e')
                           # s=64)
            
        ax.legend()
        ax.set_xlabel('Time (days)')

        return fig

In [25]:
MAX_EPOCHS = 100
MAX_PATIENCE = 3
LEARNING_RATE = 1e-3
BATCH_SIZE = 32
SHUFFLE = True

def compile_and_fit(model: tf.keras.Model,
                    wg: CryptoDataset,
                    epochs: int = MAX_EPOCHS,
                    patience: int = MAX_PATIENCE,
                    learning_rate: float = LEARNING_RATE,
                    shuffle: bool = SHUFFLE) -> tf.keras.callbacks.History:
    """
    Performs model compilation and fitting for the given model.

    :param model: tf.keras.Model to compile and fit to the training data.
    :param wg: WindowGenerator from which the training dataset is used for
               fitting.
    :param epochs: the number of epochs to train the model on the training and
                   validation data.
    :param patience: number of epochs to determine early stopping.
    :param learning_rate: learning rate to use for the optimizer.
    :param shuffle: whether to shuffle the batches before fitting the model.
    :return: tf.keras.callbacks.History from model.fit() on training dataset.
    """

    # metrics=[tf.keras.metrics.MeanAbsoluteError()]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.MeanSquaredError()) 
    
    # Regularisation methods to prevent overfitting.
    # Early stopping to prevent over-fitting, and restore the best weights when
    # returning the model.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min',
                                                      restore_best_weights=True)

    t = time.perf_counter()
    history =  model.fit(wg.training_dataset,
                         epochs=epochs,
                         validation_data=wg.validation_dataset,
                         callbacks=[early_stopping])
    t = time.perf_counter() - t
    history.training_time = t
    
    return history

# Model To Use In Back-End

We want our back-end server to serve 7 predictions based on the last 14 days. We'll choose the best model **for now** and train it.

In [26]:
def Linear_NN(model_name,
              dataset):
    label_steps = dataset.label_steps
    n_label_features = len(dataset.label_columns)
    model = tf.keras.Sequential([
        # (batch_size, input_steps, input_features)
        # Gets last time step
        tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
        # => (batch_size, 1, input_features)
        tf.keras.layers.Dense(dataset.label_steps * n_label_features),
        # => (batch_size, 1, label_steps * label_features)
        tf.keras.layers.Reshape([dataset.label_steps, n_label_features])
        # => (batch_size, label_steps, label_features)
    ])
    model.model_name = model_name
    return model

# Training, Validation, Testing, Evaluation Script

Python functions and script to use to evaluate a given set of `(coin, vs_currency)` pairs (ideally stored as a list of length-of-2 lists in a JSON object in `coin_vs_currency_pairs.json`) to be read to generate results for evaluation.

In [27]:
# Example JSON to read in
coin_vs_currency_json = {'coin_vs_currency': [['bitcoin', 'USD'],
                                              ['ethereum', 'GBP'],
                                              ['dogecoin', 'CAD']]}

# Example script input
coin_vs_currency_pairs = coin_vs_currency_json['coin_vs_currency']
coin_vs_currency_pairs

[['bitcoin', 'USD'], ['ethereum', 'GBP'], ['dogecoin', 'CAD']]

In [28]:
# TODO: Functions to use to to generate evaluation for each pair.

In [29]:
# TODO: Script to call above functions in a loop over `coin_vs_currency_pairs`

# Training & Saving Script

Python functions and script to use to train a given list of `coin_id`s against every currency in a list of `vs_currency` (ideally stored in a JSON object in `config.json`, named as such, because this is also used by the front-end to decide which coins to show the user).

We need to train a model for every pair of `coin_id` to `vs_currency`. For the below example, we have $5$ coins, and $3$ versus currencies. We want to generate a model for every coin and versus currency, therefore we need $5 \times 3 = 15$ models.

When we save the model, we'll save it with the name `f'{coin_id}_vs_{vs_currency}'` e.g. for `coin_id='bitcoin'` and `vs_currency='USD'`, the model should be saved with the name `bitcoin_vs_USD` under a directory e.g. `models/` directory.

In [30]:
# Example JSON to read in
config = {'coin_id': ['BTC',
                      'ETH',
                      'DOGE',
                      'USDT',
                      'XRP'],
          'vs_currency': ['USD',
                          'GBP',
                          'CAD']}

# Example script input
coin_vs_currency_pairs = [(coin_id, vs_currency)
                          for coin_id in config['coin_id']
                          for vs_currency in config['vs_currency']]
coin_vs_currency_pairs

[('BTC', 'USD'),
 ('BTC', 'GBP'),
 ('BTC', 'CAD'),
 ('ETH', 'USD'),
 ('ETH', 'GBP'),
 ('ETH', 'CAD'),
 ('DOGE', 'USD'),
 ('DOGE', 'GBP'),
 ('DOGE', 'CAD'),
 ('USDT', 'USD'),
 ('USDT', 'GBP'),
 ('USDT', 'CAD'),
 ('XRP', 'USD'),
 ('XRP', 'GBP'),
 ('XRP', 'CAD')]

In [31]:
histories = dict()

def generate_coin_vs_currency_model(coin_id: str,
                                    vs_currency: str,
                                    model_constructor,
                                    input_steps: int = 14,
                                    label_steps: int = 7,
                                    offset: int = 7,
                                    scale_columns: List[str] = ['Open', 'High', 'Low', 'Close', 'Volume'],
                                    label_columns: List[str] = ['Close']):
    t = time.perf_counter()
    model_name = f'{coin_id}_vs_{vs_currency}'

    # Fetch data
    df = fetch_yahoo_finance_market_chart(coin_id, vs_currency)

    # Split into training and validation pandas.DataFrames.
    split_idx = int(len(df) * 0.8)
    training_df = df[:split_idx]
    validation_df = df[split_idx:]

    # Create windowed dataset on normalised data.
    ds = CryptoDataset(training_df,
                       validation_df,
                       None,
                       input_steps=input_steps,
                       label_steps=label_steps,
                       offset=offset,
                       scale_columns=scale_columns,
                       label_columns=label_columns)

    # Construct, compile and fit model.
    model = model_constructor(model_name, ds)
    histories[model_name] = compile_and_fit(model, ds)

    # Save model.
    model.save(f'tf_models/{model_name}')
    # Save scaler too.
    with open(f'tf_models/{model_name}/zscaler.pkl', 'wb') as f:
        pkl.dump(ds.zscaler, f)

    print(f'Created {model_name} in {time.perf_counter() - t}s')

    return model, ds.zscaler

Make sure directory to save models exists.

In [32]:
!rm -rf tf_models
!mkdir -p tf_models
!ls -la

total 20
drwxr-xr-x 1 root root 4096 Mar 24 06:02 .
drwxr-xr-x 1 root root 4096 Mar 24 05:44 ..
drwxr-xr-x 4 root root 4096 Mar 22 13:38 .config
drwxr-xr-x 1 root root 4096 Mar 22 13:39 sample_data
drwxr-xr-x 2 root root 4096 Mar 24 06:02 tf_models


In [33]:
models = dict()
t = time.perf_counter()
for coin_id, vs_currency in coin_vs_currency_pairs:
    if coin_id not in models:
        models[coin_id] = dict()
    if vs_currency not in models[coin_id]:
        models[coin_id][vs_currency] = generate_coin_vs_currency_model(coin_id, vs_currency, Linear_NN)
t = time.perf_counter() - t

Epoch 1/100
46/46 [==============================] - 1s 9ms/step - loss: 2.5419 - val_loss: 0.3889
Epoch 2/100
46/46 [==============================] - 0s 7ms/step - loss: 2.1299 - val_loss: 0.3384
Epoch 3/100
46/46 [==============================] - 0s 6ms/step - loss: 1.8187 - val_loss: 0.2959
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 1.5526 - val_loss: 0.2594
Epoch 5/100
46/46 [==============================] - 0s 6ms/step - loss: 1.3230 - val_loss: 0.2279
Epoch 6/100
46/46 [==============================] - 0s 6ms/step - loss: 1.1250 - val_loss: 0.2008
Epoch 7/100
46/46 [==============================] - 0s 5ms/step - loss: 0.9548 - val_loss: 0.1776
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 0.8093 - val_loss: 0.1578
Epoch 9/100
46/46 [==============================] - 0s 6ms/step - loss: 0.6856 - val_loss: 0.1409
Epoch 10/100
46/46 [==============================] - 0s 5ms/step - loss: 0.5809 - val_loss: 0.1265
Epoch 11/

Created BTC_vs_USD in 34.668048007999914s
Epoch 1/100
46/46 [==============================] - 1s 8ms/step - loss: 2.7729 - val_loss: 0.6952
Epoch 2/100
46/46 [==============================] - 0s 7ms/step - loss: 2.3899 - val_loss: 0.6155
Epoch 3/100
46/46 [==============================] - 0s 6ms/step - loss: 2.0994 - val_loss: 0.5482
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 1.8513 - val_loss: 0.4900
Epoch 5/100
46/46 [==============================] - 0s 5ms/step - loss: 1.6361 - val_loss: 0.4392
Epoch 6/100
46/46 [==============================] - 0s 6ms/step - loss: 1.4477 - val_loss: 0.3944
Epoch 7/100
46/46 [==============================] - 0s 5ms/step - loss: 1.2819 - val_loss: 0.3548
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1354 - val_loss: 0.3196
Epoch 9/100
46/46 [==============================] - 0s 5ms/step - loss: 1.0058 - val_loss: 0.2884
Epoch 10/100
46/46 [==============================] - 0s 6ms/step -

Created BTC_vs_GBP in 32.96282070899997s
Epoch 1/100
46/46 [==============================] - 1s 7ms/step - loss: 0.9858 - val_loss: 0.1851
Epoch 2/100
46/46 [==============================] - 0s 5ms/step - loss: 0.7816 - val_loss: 0.1578
Epoch 3/100
46/46 [==============================] - 0s 5ms/step - loss: 0.6427 - val_loss: 0.1367
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 0.5344 - val_loss: 0.1197
Epoch 5/100
46/46 [==============================] - 0s 7ms/step - loss: 0.4480 - val_loss: 0.1058
Epoch 6/100
46/46 [==============================] - 0s 7ms/step - loss: 0.3778 - val_loss: 0.0942
Epoch 7/100
46/46 [==============================] - 0s 5ms/step - loss: 0.3200 - val_loss: 0.0844
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 0.2719 - val_loss: 0.0761
Epoch 9/100
46/46 [==============================] - 0s 5ms/step - loss: 0.2319 - val_loss: 0.0689
Epoch 10/100
46/46 [==============================] - 0s 5ms/step - 

Created BTC_vs_CAD in 33.54385118599998s
Epoch 1/100
46/46 [==============================] - 1s 7ms/step - loss: 3.1074 - val_loss: 0.9593
Epoch 2/100
46/46 [==============================] - 0s 6ms/step - loss: 2.7209 - val_loss: 0.8580
Epoch 3/100
46/46 [==============================] - 0s 5ms/step - loss: 2.4416 - val_loss: 0.7724
Epoch 4/100
46/46 [==============================] - 0s 6ms/step - loss: 2.2011 - val_loss: 0.6979
Epoch 5/100
46/46 [==============================] - 0s 5ms/step - loss: 1.9901 - val_loss: 0.6323
Epoch 6/100
46/46 [==============================] - 0s 7ms/step - loss: 1.8033 - val_loss: 0.5741
Epoch 7/100
46/46 [==============================] - 0s 7ms/step - loss: 1.6370 - val_loss: 0.5223
Epoch 8/100
46/46 [==============================] - 0s 6ms/step - loss: 1.4881 - val_loss: 0.4758
Epoch 9/100
46/46 [==============================] - 0s 5ms/step - loss: 1.3542 - val_loss: 0.4339
Epoch 10/100
46/46 [==============================] - 0s 5ms/step - 

Created ETH_vs_USD in 33.861069872999906s
Epoch 1/100
46/46 [==============================] - 1s 7ms/step - loss: 0.6542 - val_loss: 0.3745
Epoch 2/100
46/46 [==============================] - 0s 6ms/step - loss: 0.5066 - val_loss: 0.3100
Epoch 3/100
46/46 [==============================] - 0s 6ms/step - loss: 0.4127 - val_loss: 0.2602
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 0.3397 - val_loss: 0.2207
Epoch 5/100
46/46 [==============================] - 0s 6ms/step - loss: 0.2823 - val_loss: 0.1890
Epoch 6/100
46/46 [==============================] - 0s 5ms/step - loss: 0.2367 - val_loss: 0.1635
Epoch 7/100
46/46 [==============================] - 0s 7ms/step - loss: 0.2005 - val_loss: 0.1428
Epoch 8/100
46/46 [==============================] - 0s 6ms/step - loss: 0.1716 - val_loss: 0.1258
Epoch 9/100
46/46 [==============================] - 0s 6ms/step - loss: 0.1483 - val_loss: 0.1118
Epoch 10/100
46/46 [==============================] - 0s 5ms/step -

...

Created USDT_vs_GBP in 21.539347733999875s
Epoch 1/100
46/46 [==============================] - 1s 7ms/step - loss: 0.7783 - val_loss: 0.7344
Epoch 2/100
46/46 [==============================] - 0s 6ms/step - loss: 0.6490 - val_loss: 0.6327
Epoch 3/100
46/46 [==============================] - 0s 4ms/step - loss: 0.5505 - val_loss: 0.5491
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 0.4720 - val_loss: 0.4800
Epoch 5/100
46/46 [==============================] - 0s 6ms/step - loss: 0.4094 - val_loss: 0.4226
Epoch 6/100
46/46 [==============================] - 0s 5ms/step - loss: 0.3592 - val_loss: 0.3748
Epoch 7/100
46/46 [==============================] - 0s 6ms/step - loss: 0.3185 - val_loss: 0.3346
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 0.2852 - val_loss: 0.3006
Epoch 9/100
46/46 [==============================] - 0s 5ms/step - loss: 0.2575 - val_loss: 0.2717
Epoch 10/100
46/46 [==============================] - 0s 5ms/step 

Created USDT_vs_CAD in 33.992085193000094s
Epoch 1/100
46/46 [==============================] - 1s 8ms/step - loss: 1.1054 - val_loss: 0.1478
Epoch 2/100
46/46 [==============================] - 0s 5ms/step - loss: 0.8922 - val_loss: 0.1199
Epoch 3/100
46/46 [==============================] - 0s 6ms/step - loss: 0.7422 - val_loss: 0.0982
Epoch 4/100
46/46 [==============================] - 0s 5ms/step - loss: 0.6207 - val_loss: 0.0808
Epoch 5/100
46/46 [==============================] - 0s 6ms/step - loss: 0.5217 - val_loss: 0.0668
Epoch 6/100
46/46 [==============================] - 0s 6ms/step - loss: 0.4410 - val_loss: 0.0556
Epoch 7/100
46/46 [==============================] - 0s 5ms/step - loss: 0.3751 - val_loss: 0.0466
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 0.3214 - val_loss: 0.0394
Epoch 9/100
46/46 [==============================] - 0s 5ms/step - loss: 0.2777 - val_loss: 0.0336
Epoch 10/100
46/46 [==============================] - 0s 5ms/step 

Created XRP_vs_USD in 16.313672234000023s
Epoch 1/100
46/46 [==============================] - 1s 7ms/step - loss: 1.1702 - val_loss: 0.1359
Epoch 2/100
46/46 [==============================] - 0s 8ms/step - loss: 0.9587 - val_loss: 0.1144
Epoch 3/100
46/46 [==============================] - 0s 7ms/step - loss: 0.8088 - val_loss: 0.0972
Epoch 4/100
46/46 [==============================] - 0s 6ms/step - loss: 0.6858 - val_loss: 0.0831
Epoch 5/100
46/46 [==============================] - 0s 5ms/step - loss: 0.5839 - val_loss: 0.0715
Epoch 6/100
46/46 [==============================] - 0s 6ms/step - loss: 0.4991 - val_loss: 0.0619
Epoch 7/100
46/46 [==============================] - 0s 5ms/step - loss: 0.4285 - val_loss: 0.0540
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 0.3698 - val_loss: 0.0475
Epoch 9/100
46/46 [==============================] - 0s 5ms/step - loss: 0.3209 - val_loss: 0.0422
Epoch 10/100
46/46 [==============================] - 0s 5ms/step -

Created XRP_vs_GBP in 34.09601272500004s
Epoch 1/100
46/46 [==============================] - 1s 7ms/step - loss: 2.4555 - val_loss: 0.3348
Epoch 2/100
46/46 [==============================] - 0s 7ms/step - loss: 2.1280 - val_loss: 0.2925
Epoch 3/100
46/46 [==============================] - 0s 6ms/step - loss: 1.8785 - val_loss: 0.2566
Epoch 4/100
46/46 [==============================] - 0s 6ms/step - loss: 1.6613 - val_loss: 0.2253
Epoch 5/100
46/46 [==============================] - 0s 5ms/step - loss: 1.4698 - val_loss: 0.1977
Epoch 6/100
46/46 [==============================] - 0s 5ms/step - loss: 1.3000 - val_loss: 0.1733
Epoch 7/100
46/46 [==============================] - 0s 5ms/step - loss: 1.1489 - val_loss: 0.1516
Epoch 8/100
46/46 [==============================] - 0s 5ms/step - loss: 1.0143 - val_loss: 0.1324
Epoch 9/100
46/46 [==============================] - 0s 6ms/step - loss: 0.8945 - val_loss: 0.1155
Epoch 10/100
46/46 [==============================] - 0s 5ms/step - 

Created XRP_vs_CAD in 18.73936072299989s


In [34]:
minutes, seconds = divmod(t, 60)
hours, minutes = divmod(minutes, 60)
print(f'Trained {len(coin_vs_currency_pairs)} models in {int(hours)}:{int(minutes)}:{seconds}')
print(json.dumps(models, indent=4, default=lambda x: x.model_name if hasattr(x, 'model_name') else str(x)))

Trained 15 models in 0:7:54.763624543000105
{
    "BTC": {
        "USD": [
            "BTC_vs_USD",
            "<__main__.ZScaler object at 0x7f86d9432ee0>"
        ],
        "GBP": [
            "BTC_vs_GBP",
            "<__main__.ZScaler object at 0x7f86d4152550>"
        ],
        "CAD": [
            "BTC_vs_CAD",
            "<__main__.ZScaler object at 0x7f86d9432160>"
        ]
    },
    "ETH": {
        "USD": [
            "ETH_vs_USD",
            "<__main__.ZScaler object at 0x7f86c7ccc820>"
        ],
        "GBP": [
            "ETH_vs_GBP",
            "<__main__.ZScaler object at 0x7f86c7b2ec40>"
        ],
        "CAD": [
            "ETH_vs_CAD",
            "<__main__.ZScaler object at 0x7f86d416e910>"
        ]
    },
    "DOGE": {
        "USD": [
            "DOGE_vs_USD",
            "<__main__.ZScaler object at 0x7f86d41b92e0>"
        ],
        "GBP": [
            "DOGE_vs_GBP",
            "<__main__.ZScaler object at 0x7f86c6565ac0>"
        ],
    

## Loading Saved Models

In [35]:
!ls -la tf_models

total 68
drwxr-xr-x 17 root root 4096 Mar 24 06:10 .
drwxr-xr-x  1 root root 4096 Mar 24 06:02 ..
drwxr-xr-x  4 root root 4096 Mar 24 06:04 BTC_vs_CAD
drwxr-xr-x  4 root root 4096 Mar 24 06:03 BTC_vs_GBP
drwxr-xr-x  4 root root 4096 Mar 24 06:03 BTC_vs_USD
drwxr-xr-x  4 root root 4096 Mar 24 06:07 DOGE_vs_CAD
drwxr-xr-x  4 root root 4096 Mar 24 06:07 DOGE_vs_GBP
drwxr-xr-x  4 root root 4096 Mar 24 06:06 DOGE_vs_USD
drwxr-xr-x  4 root root 4096 Mar 24 06:05 ETH_vs_CAD
drwxr-xr-x  4 root root 4096 Mar 24 06:05 ETH_vs_GBP
drwxr-xr-x  4 root root 4096 Mar 24 06:04 ETH_vs_USD
drwxr-xr-x  4 root root 4096 Mar 24 06:09 USDT_vs_CAD
drwxr-xr-x  4 root root 4096 Mar 24 06:08 USDT_vs_GBP
drwxr-xr-x  4 root root 4096 Mar 24 06:08 USDT_vs_USD
drwxr-xr-x  4 root root 4096 Mar 24 06:10 XRP_vs_CAD
drwxr-xr-x  4 root root 4096 Mar 24 06:10 XRP_vs_GBP
drwxr-xr-x  4 root root 4096 Mar 24 06:09 XRP_vs_USD


In [36]:
!ls -la tf_models/BTC_vs_USD

total 116
drwxr-xr-x  4 root root  4096 Mar 24 06:03 .
drwxr-xr-x 17 root root  4096 Mar 24 06:10 ..
drwxr-xr-x  2 root root  4096 Mar 24 06:03 assets
-rw-r--r--  1 root root    52 Mar 24 06:03 fingerprint.pb
-rw-r--r--  1 root root  6932 Mar 24 06:03 keras_metadata.pb
-rw-r--r--  1 root root 84872 Mar 24 06:03 saved_model.pb
drwxr-xr-x  2 root root  4096 Mar 24 06:03 variables
-rw-r--r--  1 root root  1145 Mar 24 06:03 zscaler.pkl


Get the model and zscaler within this runtime to check the predicted price.

In [41]:
BTC_vs_USD, zscaler = models['BTC']['USD']
BTC_vs_USD, zscaler

(<keras.engine.sequential.Sequential at 0x7f86d4260400>,
 <__main__.ZScaler at 0x7f86d9432ee0>)

Let's generate example data to predict from.

In [42]:
df = fetch_yahoo_finance_market_chart('BTC', 'USD', period='14d')
df

,Open,High,Low,Close,Volume
Date,,,,,
2023-03-11 00:00:00+00:00,20187.876953,20792.525391,20068.660156,20632.410156,30180288176
2023-03-12 00:00:00+00:00,20628.029297,22185.031250,20448.806641,22163.949219,29279035521
2023-03-13 00:00:00+00:00,22156.406250,24550.837891,21918.199219,24197.533203,49466362688
2023-03-14 00:00:00+00:00,24201.765625,26514.716797,24081.183594,24746.074219,54622230164
2023-03-15 00:00:00+00:00,24770.925781,25240.615234,23964.910156,24375.960938,43655701450
2023-03-16 00:00:00+00:00,24373.457031,25190.326172,24225.111328,25052.789062,33866061747
2023-03-17 00:00:00+00:00,25055.123047,27787.812500,24955.169922,27423.929688,50730261335
2023-03-18 00:00:00+00:00,27448.117188,27725.953125,26636.261719,26965.878906,35723036817
2023-03-19 00:00:00+00:00,26969.503906,28440.560547,26907.716797,28038.675781,37769448859


In [43]:
# Transform data using ZScaler
inputs = zscaler.transform(df)
print(inputs.shape)
inputs

(14, 5)


array([[ 0.00807564,  0.01258753,  0.03349272,  0.03115152,  0.16637742],
       [ 0.03202298,  0.08634729,  0.05484848,  0.1144691 ,  0.1232708 ],
       [ 0.1151773 ,  0.21166191,  0.13739562,  0.22509852,  1.08882405],
       [ 0.22645904,  0.31568678,  0.25890716,  0.25493981,  1.3354275 ],
       [ 0.25742531,  0.24819878,  0.25237519,  0.2348052 ,  0.81090202],
       [ 0.23580025,  0.24553502,  0.2669927 ,  0.27162547,  0.34266676],
       [ 0.27288761,  0.3831215 ,  0.30800573,  0.40061838,  1.14927591],
       [ 0.4030831 ,  0.37984486,  0.40244565,  0.37569986,  0.43148527],
       [ 0.37704322,  0.41769697,  0.41769538,  0.43406131,  0.52936449],
       [ 0.43537277,  0.42231397,  0.43652413,  0.41929467,  0.8643912 ],
       [ 0.42050831,  0.4176441 ,  0.44757798,  0.44152192,  0.44962018],
       [ 0.44174486,  0.43691285,  0.40939676,  0.39428106,  0.31951529],
       [ 0.39513097,  0.43302003,  0.43318057,  0.45012581, -0.11868047],
       [ 0.45248407,  0.41449803,  0.4

In [44]:
inputs = inputs[np.newaxis, :, :]
print(inputs.shape)
inputs

(1, 14, 5)


array([[[ 0.00807564,  0.01258753,  0.03349272,  0.03115152,
          0.16637742],
        [ 0.03202298,  0.08634729,  0.05484848,  0.1144691 ,
          0.1232708 ],
        [ 0.1151773 ,  0.21166191,  0.13739562,  0.22509852,
          1.08882405],
        [ 0.22645904,  0.31568678,  0.25890716,  0.25493981,
          1.3354275 ],
        [ 0.25742531,  0.24819878,  0.25237519,  0.2348052 ,
          0.81090202],
        [ 0.23580025,  0.24553502,  0.2669927 ,  0.27162547,
          0.34266676],
        [ 0.27288761,  0.3831215 ,  0.30800573,  0.40061838,
          1.14927591],
        [ 0.4030831 ,  0.37984486,  0.40244565,  0.37569986,
          0.43148527],
        [ 0.37704322,  0.41769697,  0.41769538,  0.43406131,
          0.52936449],
        [ 0.43537277,  0.42231397,  0.43652413,  0.41929467,
          0.8643912 ],
        [ 0.42050831,  0.4176441 ,  0.44757798,  0.44152192,
          0.44962018],
        [ 0.44174486,  0.43691285,  0.40939676,  0.39428106,
          0.319

In [45]:
# Example prediction using a generated-and-trained model.
outputs = BTC_vs_USD.predict(inputs)
print(outputs.shape)
outputs

1/1 [==============================] - 0s 92ms/step
(1, 7, 1)


array([[[0.46221372],
        [0.45900643],
        [0.4480897 ],
        [0.44970927],
        [0.43530768],
        [0.470033  ],
        [0.44046038]]], dtype=float32)

In [46]:
# Get output as 2D array.
outputs = outputs[0]
# Inverse transform inputs using zscaler
zscaler.inverse_transform(outputs)
outputs

array([[28556.172],
       [28497.215],
       [28296.545],
       [28326.316],
       [28061.586],
       [28699.906],
       [28156.305]], dtype=float32)

In [47]:
list(outputs.flatten())

[28556.172, 28497.215, 28296.545, 28326.316, 28061.586, 28699.906, 28156.305]

In [48]:
df.tail(7)

,Open,High,Low,Close,Volume
Date,,,,,
2023-03-18 00:00:00+00:00,27448.117188,27725.953125,26636.261719,26965.878906,35723036817
2023-03-19 00:00:00+00:00,26969.503906,28440.560547,26907.716797,28038.675781,37769448859
2023-03-20 00:00:00+00:00,28041.601562,28527.724609,27242.880859,27767.236328,44774027664
2023-03-21 00:00:00+00:00,27768.392578,28439.562500,27439.646484,28175.816406,36102192830
2023-03-22 00:00:00+00:00,28158.720703,28803.335938,26759.996094,27307.437500,33382021890
2023-03-23 00:00:00+00:00,27301.957031,28729.843750,27183.363281,28333.972656,24220433689
2023-03-24 00:00:00+00:00,28356.107422,28380.167969,28153.300781,28277.394531,22958860288


In [49]:
!ls -la drive/MyDrive/tf_models/BTC_vs_USD

ls: cannot access 'drive/MyDrive/tf_models/BTC_vs_USD': No such file or directory


In [ ]:
# for coin_id, vs_currency in coin_vs_currency_pairs:
    # restored_model = tf.keras.models.load_model(f'tf_models/{coin_id}_vs_{vs_currency}')
restored_model = tf.keras.models.load_model(f'drive/MyDrive/tf_models/BTC_vs_USD')
with open(f'drive/MyDrive/tf_models/BTC_vs_USD/zscaler.pkl', 'rb') as f:
    restored_zscaler = pkl.load(f)
restored_outputs = restored_model.predict(inputs)
restored_outputs = restored_outputs[0]
restored_zscaler.inverse_transform(restored_outputs)
restored_outputs
# Yay, they match.